In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Model Architecture

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channel_img, feature_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(channel_img, feature_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            self._block(feature_d, feature_d*2, 4, 2, 1),
            self._block(feature_d*2, feature_d*4, 4, 2, 1),
            self._block(feature_d*4, feature_d*8, 4, 2, 1),
            nn.Conv2d(feature_d*8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid()
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self, x):
        return self.disc(x)
    
    

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim, channel_img, feature_g): # z -> latent vector or noise
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            self._block(z_dim, 1024, 4, 1, 0),
            self._block(1024, 512, 4, 2, 1),
            self._block(512, 256, 4, 2, 1),
            self._block(256, 128, 4, 2, 1),
            nn.ConvTranspose2d(128, channel_img, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
    

    def forward(self, x):
        return self.gen(x)

In [ ]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)


## Data Preparation

In [ ]:
LEARNING_RATE = 2e-4 
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 3
NOISE_DIM = 100
NUM_EPOCHS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64
WORKERS = 2

In [ ]:
BASE_DIR = "/home/jovyan/work/workspace/GAN-face-gen/"
IMAGE_DIR = "/home/jovyan/work/workspace/GAN-face-gen/data/face-images/"
PARTITION_FILE = "/home/jovyan/work/workspace/GAN-face-gen/data/list_eval_partition.csv"

In [ ]:
partition_file = pd.read_csv(PARTITION_FILE)

train_df = partition_file[partition_file['partition'] == 0]
val_df = partition_file[partition_file['partition'] == 1]
test_df = partition_file[partition_file['partition'] == 2]

In [ ]:
image_files = os.listdir(IMAGE_DIR)
image_files = [f for f in image_files if f.endswith(('.jpg', '.png'))]


In [ ]:
class FaceDataset(Dataset):
    def __init__(self, img_dir, data, transform=None):
        self.img_dir = img_dir
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)])
    ]
)


In [ ]:
train_loader = DataLoader(
    FaceDataset(IMAGE_DIR, train_df, transform=transforms),
    batch_size=BATCH_SIZE,
    num_workers= WORKERS,
    shuffle=True
)

val_loader = DataLoader(
    FaceDataset(IMAGE_DIR, val_df, transform=transforms),
    batch_size=BATCH_SIZE,
    num_workers= WORKERS,
    shuffle=True
)

test_loader = DataLoader(
    FaceDataset(IMAGE_DIR, test_df, transform=transforms),
    batch_size=BATCH_SIZE,
    num_workers= WORKERS,
    shuffle=True
)

In [ ]:
# def denormalize(img):
#     means = torch.tensor(normalise_means).view(3, 1, 1)
#     stds = torch.tensor(normalise_stds).view(3, 1, 1)
#     return img * stds + means

In [ ]:
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)

initialize_weights(gen)
initialize_weights(disc)


In [ ]:
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()